# Model 1 - custom MCMC

This example builds upon the previous notebook, so we'll be more terse about some of the stuff already covered, and focus on how to do posterior estimation using MCMC.

![Model1](img/model1.png)
Taken from Figure 6 of Vincent (2015).

## Define the posterior density function

In [1]:
using StatsFuns

# define Φ() as the cumulative normal distribution function
Φ(x) = normcdf.(0, 1, x)

# define the deterministic node PC, corresponds to Equation 2 in Vincent (2015)
PC(Δμ, σ2::Number) = Φ(Δμ/sqrt(2σ2));

log_prior(σ2) = log(1/1000)

function log_likelihood(σ2::Number, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Integer)
    log_likelihood = sum(binomlogpdf.(T, PC(Δμ, σ2), k))
end

log_posterior(σ2, Δμ, k, T::Integer) = log_prior(σ2) + log_likelihood(σ2, Δμ, k, T)

posterior_density(σ2, Δμ, k, T) = exp(log_posterior(σ2, Δμ, k, T::Integer))

posterior_density (generic function with 1 method)

## Define our data

In [2]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

## Define the MH Algorithm

In [3]:
using Distributions

function mhAlgorithm(pdf, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Number, θ₀::Number, nSamples=10^5, proposalσ=0.1)
    
    accepted = 0
    samples = zeros(Float64, nSamples,1)
    samples[1] = θ₀

    for n = 2:nSamples
        old_sample = samples[n-1]
        old_posterior = pdf(old_sample, Δμ, k, T)

        # suggest a new sample, but θ can't be less than or equal to zero
        new_sample = -Inf
        while new_sample ≤ 0
            new_sample = rand(Normal(old_sample,proposalσ))
        end
        new_posterior = pdf(new_sample, Δμ, k, T)

        # maybe accept new sample
        if new_posterior > old_posterior
            samples[n] = new_sample
            accepted += 1
        else
            u = rand(Normal(0,1))
            if u < new_posterior/old_posterior
                samples[n] = new_sample
                accepted += 1
            else
                samples[n] = old_sample
            end
        end

#         if rem(n,5000)==0
#             println("$n of $nSamples")
#         end
    end

    acceptance_rate = accepted / nSamples;
    println("Acceptance rate: $(acceptance_rate*100) %")
    return samples
end

mhAlgorithm (generic function with 3 methods)

# Generate samples using the MH algorithm

In [4]:
@time samples = mhAlgorithm(posterior_density, data["Δμ"], data["k"], data["T"], 2);

Acceptance rate: 85.04899999999999 %
  0.467548 seconds (788.02 k allocations: 102.231 MiB, 4.14% gc time)


In [5]:
using Plots
plot(samples, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 100000 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 3.5 
 
<polyline clip-path="url(#clip0302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,173.53 32.2276,175.25 32.2331,167.024 32.2385,175.78 32.244,160.084 32.2495,160.176 32.255,170.517 32.2605,190.234 32.266,187.566 32.2714,187.888 
 32.2769,166.903 32.2824,183.628 32.2879,184.258 32.2934,201.827 32.2988,200.069 32.3043,206.106 32.3098,208.45 32.3153,199.673 32.3208,206.678 32.3262,206.678 
 32.3317,216.054 32.3372,203.297 32.3427,206.426 32.3482,206.186 32.3536,206.186 32.3591,219.641 32.3646,219.641 32.3701,236.662 32.3756,232.554 32.3811,232.995 
 32.3865,232.995 32.392,234.777 32.3975,234.777 32.403,239.537 32.4085,232.948 32.4139,246.25 32.4194,252.706 32.4249,235.785 32.4304,247.347 32.4359,252.943 
 32.4413,251.644 32.4468,255.346 32.4523,255.972 32.4578,259.434 32.4633,258.219 32.4687,259.611 32.4742,265.767 32.4797,271.468 32.4852,270.827 32.4907,278.175 
 32.4962,289.307 32.5016,280.191 32.5071,271.5 32.5126,271.5 32.5181,271.5 32.5236,277.556 32.529,268.377 32.5345,267.961 32.54,268.182 32.5455,268.182 
 32.551,262.67 32.5564,254.246 32.5619,256.655 32.5674,265.536 32.5729,265.536 32.5784,279.536 32.5839,282.667 32.5893,282.103 32.5948,286.981 32.6003,304.581 
 32.6058,300.142 32.6113,295.801 32.6167,300.921 32.6222,291.079 32.6277,281.467 32.6332,310.267 32.6387,310.267 32.6441,286.199 32.6496,295.314 32.6551,295.314 
 32.6606,289.395 32.6661,278.191 32.6715,289.903 32.677,279.165 32.6825,270.719 32.688,270.719 32.6935,277.69 32.699,263.169 32.7044,268.279 32.7099,267.619 
 32.7154,282.031 32.7209,300.182 32.7264,288.984 32.7318,293.41 32.7373,303.125 32.7428,302.372 32.7483,298.609 32.7538,316.209 32.7592,316.209 32.7647,307.355 
 32.7702,307.355 32.7757,277.276 32.7812,277.276 32.7866,284.571 32.7921,294.307 32.7976,293.395 32.8031,293.395 32.8086,293.395 32.8141,287.116 32.8195,287.116 
 32.825,287.84 32.8305,276.684 32.836,271.732 32.8415,268.681 32.8469,255.542 32.8524,261.136 32.8579,258.291 32.8634,243.056 32.8689,243.056 32.8743,221.003 
 32.8798,234.66 32.8853,241.836 32.8908,254.561 32.8963,251.061 32.9017,238.503 32.9072,230.118 32.9127,245.546 32.9182,235.612 32.9237,226.131 32.9292,229.141 
 32.9346,236.052 32.9401,236.052 32.9456,236.052 32.9511,248.235 32.9566,246.117 32.962,247.589 32.9675,245.674 32.973,250.943 32.9785,256.77 32.984,271.494 
 32.9894,271.494 32.9949,282.937 33.0004,291.749 33.0059,303.372 33.0114,314.906 33.0168,329.424 33.0223,332.149 33.0278,332.149 33.0333,320.428 33.0388,329.172 
 33.0443,337.233 33.0497,337.233 33.0552,348.032 33.0607,336.726 33.0662,327.635 33.0717,327.635 33.0771,327.635 33.0826,315.052 33.0881,308.744 33.0936,308.744 
 33.0991,284.859 33.1045,277.696 33.11,284.085 33.1155,284.448 33.121,281.648 33.1265,284.513 33.1319,284.513 33.1374,290.9 33.1429,301.32 33.1484,322.575 
 33.1539,332.5 33.1594,322.893 33.1648,322.893 33.1703,298.872 33.1758,297.459 33.1813,281.411 33.1868,281.411 33.1922,280.882 33.1977,280.882 33.2032,273.091 
 33.2087,291.79 33.2142,291.79 33.2196,300.39 33.2251,291.148 33.2306,291.148 33.2361,291.148 33.2416,290.991 33.247,287.455 33.2525,287.455 33.258,291.713 
 33.2635,289.821 33.269,288.771 33.2745,285.77 33.2799,281.058 33.2854,267.199 33.2909,267.199 33.2964,266.188 33.3019,269.06 33.3073,266.229 33.3128,271.38 
 33.3183,271.38 33.3238,270.718 33.3293,280.569 33.3347,271.656 33.3402,278.623 33.3457,286.971 33.3512,300.62 33.3567,298.848 33.3622,280.404 33.3676,273.095 
 33.3731,266.881 33.3786,262.021 33.3841,244.268 33.3896,236.369 33.395,224.805 33.4005,212.741 33.406,202.893 33.4115,207.254 33.417,210.96 33.4224,210.96 
 33.4279,207.443 33.4334,205.311 33.4389,218.227 33.4444,225.198 33.4498,233.01 33.4553,219.965 33.4608,229.076 33.4663,22

In [6]:
# TODO - plot histogram of samples

In [7]:
# TODO - calculate summary stats of samples

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001